###pennylane first example


In [136]:
import pennylane as qml
from pennylane import numpy as np

In [137]:
dev = qml.device('default.qubit', wires = 4)

In [138]:
###each qubit is used for input
def statepreparation(x):
    qml.BasisState(x, wires=[0, 1, 2, 3])


In [139]:

def layer(W):

    ### each qubit is applied with rotation
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)

    ##applied entanglement with cnot
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

In [140]:
@qml.qnode(dev, interface="autograd")          
def circuit(weights, x):
    statepreparation(x)   #prepare the state
    for W in weights:
        layer(W)    #prepare the weight
    return qml.expval(qml.PauliZ(0))   #measure becz it is binary classification


def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias     ###define the variational classifier

In [141]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [142]:
def accuracy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [143]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)    ###calculate the cost function

In [144]:
data = np.loadtxt("data.txt")

X = np.array(data[:, :-1], requires_grad=False)
Y = np.array(data[:, -1], requires_grad=False)
# Y = Y * 2 - np.ones(len(Y))  # shift label from {0, 1} to {-1, 1}

for i in range(5):
    print("X = {}, Y = {: d}".format(X[i], int(Y[i])))
print("...")

X = [0. 0. 0. 0.], Y = -1
X = [0. 0. 0. 1.], Y =  1
X = [0. 0. 1. 0.], Y =  1
X = [0. 0. 1. 1.], Y = -1
X = [0. 1. 0. 0.], Y =  1
...


In [145]:
np.random.seed(0)
num_qubits = 4
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)
print(weights_init, bias_init)

[[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]
  [ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]]

 [[ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]
  [ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]]] 0.0


In [152]:
opt = qml.optimize.NesterovMomentumOptimizer(0.5)
batch_size = 5
weights = weights_init
bias = bias_init
max_weight =weights.copy()
max_bias = bias.copy()
max_acc = 0
for it in range(30):
    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)   ##update the weights and bias
    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]   ###calculate the accuracy
    acc = accuracy(Y, predictions)
    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )
    if acc > max_acc:
        max_acc = acc
        max_weight = weights.copy()
        max_bias = bias.copy()
        # print('max_acc',max_acc)

print(
        "Final weights and bias with Cost and accuracy {:0.7f} | Accuracy: {:0.7f} ".format(
            cost(max_weight, max_bias, X, Y), max_acc
        )
    )

Iter:     1 | Cost: 1.8902228 | Accuracy: 0.5000000 
Iter:     2 | Cost: 1.8650652 | Accuracy: 0.5000000 
Iter:     3 | Cost: 1.9524107 | Accuracy: 0.5000000 
Iter:     4 | Cost: 1.9523922 | Accuracy: 0.5000000 
Iter:     5 | Cost: 0.6124706 | Accuracy: 0.8333333 
Iter:     6 | Cost: 1.3013395 | Accuracy: 0.5000000 
Iter:     7 | Cost: 1.2001988 | Accuracy: 0.8333333 
Iter:     8 | Cost: 1.0289143 | Accuracy: 0.6666667 
Iter:     9 | Cost: 1.4134227 | Accuracy: 0.5000000 
Iter:    10 | Cost: 1.2598158 | Accuracy: 0.5000000 
Iter:    11 | Cost: 3.0896489 | Accuracy: 0.5000000 
Iter:    12 | Cost: 1.1259599 | Accuracy: 0.6666667 
Iter:    13 | Cost: 1.3754625 | Accuracy: 0.5000000 
Iter:    14 | Cost: 0.9407202 | Accuracy: 0.5000000 
Iter:    15 | Cost: 1.0019464 | Accuracy: 0.5000000 
Iter:    16 | Cost: 1.2474381 | Accuracy: 0.5000000 
Iter:    17 | Cost: 0.8409135 | Accuracy: 0.8333333 
Iter:    18 | Cost: 0.6170311 | Accuracy: 0.8333333 
Iter:    19 | Cost: 0.6807921 | Accuracy: 0.66